In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
os.chdir('/Users/bjarnihaukurbjarnason/Documents/genki-signals')

from genki_signals.buffers import DataBuffer

In [3]:
import time
from threading import Thread

def run_source(source, buffer):
    source.start()
    while source.is_active:
        buffer.extend(source.read())
        time.sleep(1 / 25)

In [4]:
from genki_signals.data_sources import CameraDataSource, Sampler

buffer = DataBuffer(maxlen=1)

resolution = (480, 360)

camera = CameraDataSource(resolution=resolution)
source = Sampler({"camera": camera}, sample_rate=29)

t = Thread(target=run_source, args=(source, buffer))
t.start()

In [5]:
buffer.keys()

dict_keys(['timestamp', 'image'])

In [6]:
buffer["image"].shape

(3, 480, 360, 1)

In [7]:
buffer.plot("image", plot_type="video")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [8]:
source.stop(); t.join()

In [9]:
import genki_signals.signals as s
from genki_signals.system import System

derived = [
    s.Scale("image", 2, "image_scaled"),
]

buffer = DataBuffer(maxlen=1)
system = System(source, derived)

In [10]:
t = Thread(target=run_source, args=(system, buffer))
t.start()

In [11]:
buffer.keys()

dict_keys(['timestamp', 'image', 'image_scaled', 'image_reshaped'])

In [12]:
buffer.plot("image", plot_type="video")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [13]:
buffer.plot("image_scaled", plot_type="video")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [14]:
buffer.plot("image_reshaped", plot_type="video")

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [15]:
system.stop(); t.join()

: 

: 